# Bronze.csv

Tylko energia grupowana po regionie i czasie. LSTM, Dense, XGB

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf
import xgboost as xgb
warnings.filterwarnings('ignore')

2023-09-23 02:57:45.686911: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-23 02:57:46.824054: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/majkel/miniconda3/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
df = pd.read_csv('../../data/data_world.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,1930-12-08T08:01:02.000Z,23.261,120.277,15.0,6.3,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1930-12-03T18:51:47.000Z,18.233,96.298,10.0,7.4,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1930-12-02T07:01:30.000Z,25.854,98.356,35.0,6.2,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1930-11-28T07:32:56.000Z,18.779,-106.767,15.0,6.3,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1930-11-25T19:02:53.000Z,35.050,139.129,15.0,6.9,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df[["time", "latitude", "longitude", "mag"]]

In [4]:
df["time"] = pd.to_datetime(df["time"], format="%Y-%m-%dT%H:%M:%S.%fZ")
df

,time,latitude,longitude,mag
0,1930-12-08 08:01:02.000,23.2610,120.2770,6.3
1,1930-12-03 18:51:47.000,18.2330,96.2980,7.4
2,1930-12-02 07:01:30.000,25.8540,98.3560,6.2
3,1930-11-28 07:32:56.000,18.7790,-106.7670,6.3
4,1930-11-25 19:02:53.000,35.0500,139.1290,6.9
...,...,...,...,...
797041,2018-09-01 01:14:38.230,-30.4830,-177.9279,4.3
797042,2018-09-01 01:07:59.120,-10.7558,124.3621,4.0
797043,2018-09-01 01:00:13.810,-5.5167,147.1735,4.6
797044,2018-09-01 00:27:11.440,46.8819,155.6566,4.3


In [5]:
time_cut = dt.datetime(1973, 1, 1)
df = df[df["time"] > time_cut]
df["energy"] = 10**(1.44*df["mag"]+5.24)
df["label"] = np.where(df["mag"] > 5, 1, 0)

In [6]:
time_split = 30

In [7]:
# group the data into 30 days intervals
df["time"] = df["time"].dt.floor('d')
df["time"] = df["time"] - np.min(df["time"])
df["time"] = df["time"].dt.days // time_split
df["time"] = df["time"].astype(int)

In [8]:
geo_split = 10

In [9]:
df["latitude"] = df["latitude"] // geo_split
df["latitude"] = df["latitude"] + np.abs(np.min(df["latitude"]))

df["longitude"] = df["longitude"] // geo_split
df["longitude"] = df["longitude"] + np.abs(np.min(df["longitude"]))

In [10]:
df

,time,latitude,longitude,mag,energy,label
21801,12,7.0,34.0,4.5,5.248075e+11,0
21802,12,6.0,11.0,4.4,3.767038e+11,0
21803,12,7.0,0.0,5.1,3.837072e+12,1
21804,12,7.0,34.0,4.4,3.767038e+11,0
21805,12,6.0,34.0,4.6,7.311391e+11,0
...,...,...,...,...,...,...
797041,555,5.0,0.0,4.3,2.703958e+11,0
797042,555,7.0,30.0,4.0,1.000000e+11,0
797043,555,8.0,32.0,4.6,7.311391e+11,0
797044,555,13.0,33.0,4.3,2.703958e+11,0


In [11]:
df_agg = df.groupby(["latitude", "longitude", "time"]).agg({"energy": "sum", "label": "sum"}).reset_index()
df_agg

,latitude,longitude,time,energy,label
0,0.0,13.0,268,1.018591e+12,0
1,0.0,22.0,482,3.767038e+11,0
2,0.0,31.0,375,5.248075e+11,0
3,0.0,31.0,473,7.311391e+11,0
4,0.0,33.0,280,2.703958e+11,0
...,...,...,...,...,...
71216,17.0,30.0,487,1.018591e+12,0
71217,17.0,30.0,489,3.767038e+11,0
71218,17.0,30.0,509,7.311391e+11,0
71219,17.0,30.0,538,3.767038e+11,0


In [12]:
df_agg["label"] = np.where(df_agg["label"] > 0, 1, 0)

In [32]:
def make_ds(df_agg, block_size):
    dfs_train, dfs_val = [], []
    for i in df_agg["latitude"].unique():
        for j in df_agg["longitude"].unique():
            tmp = df_agg[(df_agg["latitude"] == i) & (df_agg["longitude"] == j)]
            if not tmp.empty:
                start = max(tmp["time"].min() - block_size, 0)
                end = min(tmp["time"].max() + block_size, df_agg["time"].max())
                tmp = tmp.set_index("time").reindex(range(start, end)).fillna(0).rename_axis('time').reset_index()
                tmp["label"] = tmp["label"].shift(-1)
                tmp["latitude"] = i
                tmp["longitude"] = j
                n = int(0.8 * len(tmp))
                df_train = tmp[:n]
                df_val = tmp[n:]
                scaler = MinMaxScaler()
                df_train["energy"] = scaler.fit_transform(df_train[["energy"]])
                df_val["energy"] = scaler.transform(df_val[["energy"]])
                for idx in range(1, block_size):
                    df_train["energy" + str(idx)] = df_train["energy"].shift(idx)
                    df_val["energy" + str(idx)] = df_val["energy"].shift(idx)
                dfs_train.append(df_train)
                dfs_val.append(df_val)
    df_final_train = pd.concat(dfs_train)
    df_final_val = pd.concat(dfs_val)
    return df_final_train, df_final_val

In [33]:
block_size = 16
df_train, df_val = make_ds(df_agg, block_size)

In [34]:
df_train.dropna(inplace=True)
df_val.dropna(inplace=True)

In [67]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, return_sequences=True), input_shape=(block_size, 1)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(2),
])

In [68]:
model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy', tf.keras.metrics.F1Score()])

In [69]:
x_train = df_train.drop(["time", "label", "latitude", "longitude"], axis=1).to_numpy()
x_train = x_train.reshape(-1, block_size, 1)
y_train = df_train["label"].to_numpy().reshape(-1, 1)
x_val = df_val.drop(["time", "label", "latitude", "longitude"], axis=1).to_numpy()
x_val = x_val.reshape(-1, block_size, 1)
y_val = df_val["label"].to_numpy().reshape(-1, 1)

In [70]:
# add anoother column whith 1 - label
y_train = np.concatenate((y_train, 1 - y_train), axis=1)
y_val = np.concatenate((y_val, 1 - y_val), axis=1)

In [72]:
model.fit(x_train, y_train, epochs=20, batch_size=512, validation_data=(x_val, y_val))

Epoch 1/20
329/329 [==============================] - 47s 142ms/step - loss: 0.2758 - accuracy: 0.8933 - f1_score: 0.6374 - val_loss: 0.3143 - val_accuracy: 0.8826 - val_f1_score: 0.5753
Epoch 2/20
329/329 [==============================] - 63s 193ms/step - loss: 0.2736 - accuracy: 0.8933 - f1_score: 0.6339 - val_loss: 0.2766 - val_accuracy: 0.8888 - val_f1_score: 0.6430
Epoch 3/20
329/329 [==============================] - 82s 251ms/step - loss: 0.2673 - accuracy: 0.8947 - f1_score: 0.6456 - val_loss: 0.2789 - val_accuracy: 0.8824 - val_f1_score: 0.6733
Epoch 4/20
329/329 [==============================] - 66s 201ms/step - loss: 0.2665 - accuracy: 0.8949 - f1_score: 0.6504 - val_loss: 0.2746 - val_accuracy: 0.8898 - val_f1_score: 0.6321
Epoch 5/20
329/329 [==============================] - 75s 229ms/step - loss: 0.2639 - accuracy: 0.8958 - f1_score: 0.6554 - val_loss: 0.2746 - val_accuracy: 0.8893 - val_f1_score: 0.6574
Epoch 6/20
329/329 [==============================] - 86s 263ms/s

In [75]:
x_train = df_train.drop(["time", "label", "latitude", "longitude"], axis=1).to_numpy()
y_train = df_train["label"].to_numpy().reshape(-1, 1)
x_val = df_val.drop(["time", "label", "latitude", "longitude"], axis=1).to_numpy()
y_val = df_val["label"].to_numpy().reshape(-1, 1)

In [79]:
# add anoother column whith 1 - label
y_train = np.concatenate((y_train, 1 - y_train), axis=1)
y_val = np.concatenate((y_val, 1 - y_val), axis=1)

In [80]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(block_size, activation='relu', input_shape=(block_size, )),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2),
])

In [81]:
model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy', tf.keras.metrics.F1Score()])

In [84]:
model.fit(x_train, y_train, epochs=20, batch_size=512, validation_data=(x_val, y_val))

Epoch 1/20
329/329 [==============================] - 6s 18ms/step - loss: 0.2928 - accuracy: 0.8852 - f1_score: 0.4703 - val_loss: 0.3059 - val_accuracy: 0.8797 - val_f1_score: 0.4680
Epoch 2/20
329/329 [==============================] - 5s 16ms/step - loss: 0.2912 - accuracy: 0.8851 - f1_score: 0.4703 - val_loss: 0.3076 - val_accuracy: 0.8797 - val_f1_score: 0.4680
Epoch 3/20
329/329 [==============================] - 5s 16ms/step - loss: 0.2921 - accuracy: 0.8851 - f1_score: 0.4701 - val_loss: 0.3059 - val_accuracy: 0.8797 - val_f1_score: 0.4680
Epoch 4/20
329/329 [==============================] - 5s 16ms/step - loss: 0.2911 - accuracy: 0.8852 - f1_score: 0.4703 - val_loss: 0.3091 - val_accuracy: 0.8796 - val_f1_score: 0.4680
Epoch 5/20
329/329 [==============================] - 5s 15ms/step - loss: 0.2919 - accuracy: 0.8852 - f1_score: 0.4707 - val_loss: 0.3126 - val_accuracy: 0.8797 - val_f1_score: 0.4680
Epoch 6/20
329/329 [==============================] - 5s 15ms/step - loss: 

In [86]:
x_train = df_train.drop(["time", "label", "latitude", "longitude"], axis=1).to_numpy()
y_train = df_train["label"].to_numpy().reshape(-1, 1)
x_val = df_val.drop(["time", "label", "latitude", "longitude"], axis=1).to_numpy()
y_val = df_val["label"].to_numpy().reshape(-1, 1)

In [87]:
model = xgb.XGBClassifier(n_estimators=1000, max_depth=10, learning_rate=0.01, n_jobs=-1)

In [88]:
model.fit(x_train, y_train, eval_set=[(x_val, y_val)], eval_metric='logloss', early_stopping_rounds=10)

[0]	validation_0-logloss:0.68654
[1]	validation_0-logloss:0.68007
[2]	validation_0-logloss:0.67372
[3]	validation_0-logloss:0.66750
[4]	validation_0-logloss:0.66138
[5]	validation_0-logloss:0.65537
[6]	validation_0-logloss:0.64949
[7]	validation_0-logloss:0.64371
[8]	validation_0-logloss:0.63804
[9]	validation_0-logloss:0.63247
[10]	validation_0-logloss:0.62700
[11]	validation_0-logloss:0.62163
[12]	validation_0-logloss:0.61636
[13]	validation_0-logloss:0.61116
[14]	validation_0-logloss:0.60607
[15]	validation_0-logloss:0.60106
[16]	validation_0-logloss:0.59615
[17]	validation_0-logloss:0.59132
[18]	validation_0-logloss:0.58656
[19]	validation_0-logloss:0.58189
[20]	validation_0-logloss:0.57731
[21]	validation_0-logloss:0.57280
[22]	validation_0-logloss:0.56836
[23]	validation_0-logloss:0.56399
[24]	validation_0-logloss:0.55970
[25]	validation_0-logloss:0.55548
[26]	validation_0-logloss:0.55133
[27]	validation_0-logloss:0.54724
[28]	validation_0-logloss:0.54321
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [90]:
y_pred = model.predict(x_val)

In [91]:
accuracy_score(y_val, y_pred), f1_score(y_val, y_pred)

(0.8938714841070203, 0.4426564188783475)